Based on https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/03-advanced/generative_adversarial_network/main.py

In [1]:
import torch

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import torch.nn as nn

# DO CONVNETS.

image_size = 28 * 28 * 4
hidden_size = 512
latent_size = 64

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

In [3]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [4]:
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()


In [25]:
import h5py
from torch.utils.data.dataset import Dataset
from torchvision import transforms
import numpy as np

class H5Dataset(Dataset):
    def __init__(self, fname, dname, transforms=None):
        self.f = h5py.File(fname, 'r')
        self.dataset = self.f[dname]
        self.transforms = transforms
    def __getitem__(self, key):
        datum = self.dataset[key]
        if self.transforms is not None:
            datum = self.transforms(datum)
        return datum
    def __len__(self):
        return len(self.dataset)
    def __del__(self):
        self.f.close()

# Call the dataset
data_loader = torch.utils.data.DataLoader(dataset=H5Dataset('dataset.hdf5', 'twitch_emotes', transforms.ToTensor()),
                                         batch_size=100,
                                         shuffle=False)

In [28]:
num_epochs = 200

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, images in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        # if (i+1) % 200 == 0:
        print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save the model checkpoints 
    torch.save(G.state_dict(), 'G.ckpt')
    torch.save(D.state_dict(), 'D.ckpt')
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 4, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 4, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))


Epoch [0/200], Step [1/3000], d_loss: 0.9455, g_loss: 0.6386, D(x): 0.99, D(G(z)): 0.61
Epoch [0/200], Step [2/3000], d_loss: 0.8963, g_loss: 0.6963, D(x): 0.99, D(G(z)): 0.59
Epoch [0/200], Step [3/3000], d_loss: 0.8343, g_loss: 0.7886, D(x): 0.98, D(G(z)): 0.56
Epoch [0/200], Step [4/3000], d_loss: 0.7556, g_loss: 0.9073, D(x): 0.97, D(G(z)): 0.52
Epoch [0/200], Step [5/3000], d_loss: 0.6665, g_loss: 1.0676, D(x): 0.96, D(G(z)): 0.46
Epoch [0/200], Step [6/3000], d_loss: 0.6278, g_loss: 1.2420, D(x): 0.90, D(G(z)): 0.40
Epoch [0/200], Step [7/3000], d_loss: 0.6661, g_loss: 1.3410, D(x): 0.79, D(G(z)): 0.35
Epoch [0/200], Step [8/3000], d_loss: 0.7077, g_loss: 1.2901, D(x): 0.73, D(G(z)): 0.33
Epoch [0/200], Step [9/3000], d_loss: 0.7515, g_loss: 1.1091, D(x): 0.75, D(G(z)): 0.37
Epoch [0/200], Step [10/3000], d_loss: 0.8066, g_loss: 0.9092, D(x): 0.84, D(G(z)): 0.47
Epoch [0/200], Step [11/3000], d_loss: 0.9804, g_loss: 0.7735, D(x): 0.87, D(G(z)): 0.57
Epoch [0/200], Step [12/3000],

RuntimeError: shape '[100, 1, 28, 28]' is invalid for input of size 313600